[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

In [14]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API


In [15]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [16]:
 pip install uk-covid19


Note: you may need to restart the kernel to use updated packages.


In [17]:
England_filters = [
    'areaType=nation',
    'areaName=England'
]

England_structure = {
    "date": "date",
    "cases": "newCasesBySpecimenDate",
    "new_first_episodes": "newFirstEpisodesBySpecimenDate",
    "new_reinfection": "newReinfectionsBySpecimenDate"    
}

graph = Cov19API(filters = England_filters, structure = England_structure)

In [18]:
analysis = graph.get_json()
with open("graph.json", "wt") as file:
    json.dump(analysis, file)

In [19]:
def wrangle_data(rawdata, columns_name):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    with open(rawdata, "rt") as INFILE:
        input_data = json.load(INFILE)
    
    global datalist 
    datalist = input_data['data']
    
    date_index() 
    global x_index
    x_index = pd.date_range(startdate, enddate) 
    
    global dataframe
    dataframe = pd.DataFrame(index = x_index, columns = columns_name)
    
    for i in datalist: 
        date = parse_date(i['date'])
        for x in columns_name:
            if pd.isna(dataframe.loc[date, x]): 
                value = float(i[x]) if i[x]!=None else 0.0
                dataframe.loc[date, x] = value    
                
    dataframe.fillna(0.0, inplace=True)

    return dataframe

def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def date_index():      
    dates = [i['date'] for i in datalist]
    dates.sort() 
    global startdate
    global enddate
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])

In [20]:
graph_df = wrangle_data("graph.json", ['cases', 'new_first_episodes', 'new_reinfection']) 

In [21]:
def access_api():
    return {} 

In [22]:
def api_button_callback(button):
    apidata=access_api() 
    
    global dataframe
    dataframe = wrangle_data("graph.json", ['cases', 'new_first_episodes', 'new_reinfection'])

    refresh_graph()
    apibutton.icon = "check"
    apibutton.disabled = True 
    

In [23]:
apibutton = wdg.Button(
    description = 'Refresh',
    disabled = False,
    button_style = 'info', 
    icon = 'download'
)

apibutton.on_click(api_button_callback)


In [24]:
def refresh_graph(): 
    current = scale.value
    if current == scale.options[0]:
        other = scale.options[1]
    else:
        other = scale.options[0]
    scale.value=other 
    scale.value=current    

def plot_graph(gscale, gcols):
    if gscale == 'linear':
        logscale = False
    else:
        logscale = True
        
    ncols = len(gcols)

    if ncols > 0:
        dataframe[list(gcols)].plot(logy=logscale, ylabel = 'number of', xlabel = 'time')
        plt.show()   
    else:
        print("Click to select data for graph")
        print("CTRL-Click to select more than one category")

In [25]:
series = wdg.SelectMultiple(
    options = ['cases', 'new_first_episodes', 'new_reinfection'],
    value = ['cases', 'new_first_episodes', 'new_reinfection'],
    rows = 3,
    description = 'Number of:',
    disabled = False
)


scale = wdg.RadioButtons(
    options = ['linear', 'log'],
    value = 'linear',
    description = 'Scale:',
    disabled = False)

control = wdg.HBox([series, scale])

    

## Graph of Cases, First Episode and New Infection of Covid in England
The following graph shows the number of cases, first episode and new infection of covid in England. 

1. Choose from the selection box for what statistics to show
2. Choose the scale for y-axis
3. Click the 'Refresh' box to get the latest data (optional) 

In [26]:
graph = wdg.interactive_output(plot_graph, {'gscale':scale, 'gcols': series})
display(control, apibutton, graph)

Button(button_style='info', description='Refresh', icon='download', style=ButtonStyle())

Output()

Data from Public Health England, UK Government 